# Курс "Рекомендательные системы"

## Практическое задание урока 2. Бейзлайны и детерминированные алгоритмы item-item

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
from pathlib import Path

In [3]:
DATA_ROOT = Path('./data/')

# input
DATASET_PATH = DATA_ROOT / 'retail_train.csv'

In [4]:
data = pd.read_csv(DATASET_PATH)
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
# возьмем последние 3 недели в качестве теста
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекомендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямо пропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

Сделайте предсказание.

***Решение***

In [6]:
# Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


#### 1. Random recommendation

In [7]:
def random_recommendation(items, n=5):
    """Случайные рекомендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

Сделаем предсказание

In [8]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 8.06 s


#### 2. Weighted Random Recommendation

In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные случайные рекомендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].to_list())
        
    return recs.tolist()

Сделаем предсказание

In [10]:
def get_items_weights(df):
    """Пример трех весов"""
    
    # Вес по логарифму продаж
    items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
    items_weights = items_weights[items_weights['sales_value'] >= 1]
    items_weights['log_sales_value'] = np.log(items_weights['sales_value'])
    items_weights['weight'] = items_weights['log_sales_value'] / items_weights['log_sales_value'].sum()
    items_weights = items_weights[['item_id', 'weight']]
    
    # Вес по частоте покупки
    items_weights_1 = data_train.groupby('item_id')['user_id'].unique().reset_index()
    items_weights_1['weight'] =  items_weights_1['user_id'].apply(lambda x: len(x)) 
    del items_weights_1['user_id']
    items_weights_1['weight'] /= items_weights_1.weight.sum()
    
    # Вес по (частота покупки * цена)
    items_weights_2 = data_train.groupby('item_id')['sales_value'].sum().reset_index()
    items_weights_2['weight'] =  items_weights_2['sales_value'] / items_weights_2['sales_value'].sum()
    del items_weights_2['sales_value']
    
    return items_weights, items_weights_1, items_weights_2  

In [11]:
%%time

items_weights, items_weights_1, items_weights_2 = get_items_weights(data_train)

Wall time: 18.5 s


In [12]:
%%time

# Взвешенные случайные рекомендации по логарифму продаж
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

Wall time: 21.4 s


In [13]:
%%time

# Взвешенные случайные рекомендации по частоте покупки
result['weighted_random_recommendation_1'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_1, n=5))

Wall time: 21.6 s


In [14]:
%%time

# Взвешенные случайные рекомендации по (частота покупки * цена)
result['weighted_random_recommendation_2'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_2, n=5))

Wall time: 22.6 s


In [15]:
result.head(5)

,user_id,actual,random_recommendation,weighted_random_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9885327, 15483572, 13095323, 1140726, 916588]","[1003054, 12487302, 1087581, 853756, 13987084]","[1112491, 15596957, 999207, 9858622, 933637]","[893501, 6534178, 966014, 1110764, 9553335]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6042544, 12324841, 843820, 933822, 1930655]","[9420146, 861819, 1121624, 12132014, 1971373]","[1136988, 1022527, 5568509, 866573, 12263279]","[892008, 1050082, 5568995, 959830, 854354]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[923964, 1010703, 965860, 1188488, 1584760]","[9677305, 8069355, 15921278, 930602, 5565281]","[1022872, 7144131, 945652, 987480, 820165]","[979792, 1076571, 1094955, 6534178, 12301405]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[985445, 1108221, 918703, 762022, 854478]","[6632589, 883202, 1137056, 891405, 856424]","[1119075, 1120085, 821316, 919534, 976065]","[831809, 5585639, 862104, 985119, 877287]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1021181, 6633017, 1973338, 1024500, 2423838]","[1012363, 1093625, 1102250, 12138207, 857336]","[1018740, 835278, 6773230, 878710, 885447]","[863762, 9297309, 9858960, 996070, 877202]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [16]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

***Решение***

In [17]:
result_webinar = pd.read_csv('preds.csv')# загрузка predict с семианара
result_webinar.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9267149, 8090999, 7144475, 1151239, 6772974]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7166643, 846754, 12457134, 2501172, 9652107]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [18]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result_webinar.columns[1:]:
    result_webinar[a]=  result_webinar[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [19]:
prec_random = result_webinar.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()
prec_popular = result_webinar.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()
prec_itemitem = result_webinar.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()
prec_cosine = result_webinar.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()
prec_tfidf = result_webinar.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()
prec_own_purchases = result_webinar.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()

print('precision@5 for random_recommendation:', '%.3f' % prec_random)
print('precision@5 for popular_recommendation:', '%.3f' % prec_popular)
print('precision@5 for itemitem:', '%.3f' % prec_itemitem)
print('precision@5 for cosine:', '%.3f' % prec_cosine)
print('precision@5 for tfidf:', '%.3f' % prec_tfidf)
print('precision@5 for own_purchases:', '%.3f' % prec_own_purchases)

precision@5 for random_recommendation: 0.001
precision@5 for popular_recommendation: 0.155
precision@5 for itemitem: 0.137
precision@5 for cosine: 0.133
precision@5 for tfidf: 0.139
precision@5 for own_purchases: 0.180


In [20]:
prec_weight_random = result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()
prec_weight_random_1 = result.apply(lambda row: precision_at_k(row['weighted_random_recommendation_1'], row['actual']), axis=1).mean()
prec_weight_random_2 = result.apply(lambda row: precision_at_k(row['weighted_random_recommendation_2'], row['actual']), axis=1).mean()

print('precision@5 for weighted_random_recommendation:', '%.3f' % prec_weight_random)
print('precision@5 for weighted_random_recommendation_1:', '%.3f' % prec_weight_random_1)
print('precision@5 for weighted_random_recommendation_2:', '%.3f' % prec_weight_random_2)

precision@5 for weighted_random_recommendation: 0.001
precision@5 for weighted_random_recommendation_1: 0.008
precision@5 for weighted_random_recommendation_2: 0.021


***Вывод***

Случайные рекомендации и взвешенные случайные рекомендации по логарифму продаж показывают худшее качество метрики Precision@5 (0.001).

Рекомендации на основе собственных покупок показывают лучшее качество метрики Precision@5 (0.18).

Предполагаю, что в рассматриваемом датафрейме представлены покупки товаров повседневного спроса. Такие товары пользователь покупает снова и снова, не резко изменяя привычки. Будущее поведение пользователя в большей мере обсуловлено его личным предыдущим поведением, чем поведением других потребителей. Следовательно, алгоритмы с наилучшими метриками нерационально применять для рекомендации. Лучше было бы протестировать алгортимы с меньшим качеством Precision@5 и посмотреть результаты по бизнес-метрике.

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов

In [21]:
# Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)
result_5k = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_5k.columns=['user_id', 'actual']
result_5k['actual'] = result_5k['actual'].apply(lambda x: list(x))
result_5k.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [22]:
print(data_train.item_id.unique().shape)

(86865,)


In [23]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [24]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# ~np.array([True, False])
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100
np.sort(data.item_id.unique())

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556],
      dtype=int64)

In [26]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [27]:
print(data_train.item_id.unique().shape)

(5001,)


#### 1. Улучшение бейзлайнов Weighted Random Recommendation на топ-5000 товаров

In [28]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные случайные рекомендации"""
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].to_list())
        
    return recs.tolist()

In [29]:
%%time

items_weights, items_weights_1, items_weights_2 = get_items_weights(data_train)

Wall time: 1.68 s


In [30]:
%%time

# Взвешенные случайные рекомендации по логарифму продаж
result_5k['weighted_random_recommendation'] = result_5k['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

Wall time: 1.57 s


In [31]:
%%time

# Взвешенные случайные рекомендации по частоте покупки
result_5k['weighted_random_recommendation_1'] = result_5k['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_1, n=5))

Wall time: 1.57 s


In [32]:
%%time

# Взвешенные случайные рекомендации по (частота покупки * цена)
result_5k['weighted_random_recommendation_2'] = result_5k['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_2, n=5))

Wall time: 1.73 s


In [33]:
result_5k.head(5)

,user_id,actual,weighted_random_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8019968, 1081177, 922636, 835595, 848268]","[1082212, 1026094, 5580793, 976998, 837098]","[999999, 928754, 982469, 1086641, 887984]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[902094, 1007911, 9707137, 844054, 819845]","[1046545, 1025611, 1013770, 879755, 970119]","[1021133, 866211, 999999, 1089025, 8090536]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1043920, 983959, 1022053, 961792, 1025641]","[5592993, 930603, 860975, 866573, 8020001]","[8090537, 6534178, 999999, 835098, 1029743]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[982393, 1065380, 866950, 874617, 5571178]","[5569230, 908145, 1043787, 857697, 13071902]","[999999, 6534178, 853262, 6534578, 1127179]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[826790, 1070898, 1098248, 828867, 1125197]","[944139, 1131438, 5981072, 889509, 930917]","[999999, 903738, 5996227, 10150194, 856335]"


In [34]:
prec_weight_random = result_5k.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()
prec_weight_random_1 = result_5k.apply(lambda row: precision_at_k(row['weighted_random_recommendation_1'], row['actual']), axis=1).mean()
prec_weight_random_2 = result_5k.apply(lambda row: precision_at_k(row['weighted_random_recommendation_2'], row['actual']), axis=1).mean()

print('precision@5 for weighted_random_recommendation:', '%.3f' % prec_weight_random)
print('precision@5 for weighted_random_recommendation_1:', '%.3f' % prec_weight_random_1)
print('precision@5 for weighted_random_recommendation_2:', '%.3f' % prec_weight_random_2)

precision@5 for weighted_random_recommendation: 0.008
precision@5 for weighted_random_recommendation_1: 0.016
precision@5 for weighted_random_recommendation_2: 0.030


#### 2. Улучшение ItemItemRecommender, выбирая число соседей 𝐾.

In [35]:
%%time
scores = {}
for i in range(1,10):
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)
    result_5k['itemitem'] = result_5k['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    scores[str(i)] = result_5k.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()

Wall time: 16.4 s


In [36]:
metrics = pd.DataFrame(scores.items(), columns=['Ki', 'socre'])
metrics

,Ki,socre
0,1,0.179987
1,2,0.192009
2,3,0.186092
3,4,0.144956
4,5,0.136925
5,6,0.142018
6,7,0.144858
7,8,0.147209
8,9,0.148482


#### 3. Стратегии ансамблирования изученных алгоритмов

##### 3.1. Popularity Recommendation и Random Recommendation на топ-5000 товаров

In [44]:
def popularity_recommendation(data, n=5):
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    return recs.tolist()

def random_recommendation(items, n=5):
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    return recs.tolist()

In [45]:
popular_recs = popularity_recommendation(data_train, n=5)
result_5k['popular_recommendation'] = result_5k['user_id'].apply(lambda x: popular_recs)

items = data_train.item_id.unique()
result_5k['random_recommendation'] = result_5k['user_id'].apply(lambda x: random_recommendation(items, n=5))

##### 3.2. ItemItemRecommender на топ-5000 товаров

In [46]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

itemitem_recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 1.56 s


In [40]:
%%time

result_5k['itemitem'] = result_5k['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 126 ms


##### 3.3. CosineRecommender на топ-5000 товаров

In [41]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

cosine_recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.63 s


In [42]:
%%time

result_5k['cosine'] = result_5k['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 150 ms


In [43]:
result_5k.head(5)

,user_id,actual,weighted_random_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2,itemitem,popular_recommendation,random_recommendation,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8019968, 1081177, 922636, 835595, 848268]","[1082212, 1026094, 5580793, 976998, 837098]","[999999, 928754, 982469, 1086641, 887984]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 6534178, 6533889, 1029743, 6534166]","[1133556, 1039589, 1054539, 982393, 831815]","[1082185, 999999, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[902094, 1007911, 9707137, 844054, 819845]","[1046545, 1025611, 1013770, 879755, 970119]","[1021133, 866211, 999999, 1089025, 8090536]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 6534178, 6533889, 1029743, 6534166]","[900418, 1126699, 865156, 1110255, 12988031]","[1082185, 1098066, 981760, 999999, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1043920, 983959, 1022053, 961792, 1025641]","[5592993, 930603, 860975, 866573, 8020001]","[8090537, 6534178, 999999, 835098, 1029743]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 6534178, 6533889, 1029743, 6534166]","[1128698, 825753, 1080429, 1057377, 950206]","[1082185, 999999, 981760, 1127831, 1098066]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[982393, 1065380, 866950, 874617, 5571178]","[5569230, 908145, 1043787, 857697, 13071902]","[999999, 6534178, 853262, 6534578, 1127179]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 6534178, 6533889, 1029743, 6534166]","[5995281, 1082212, 6773016, 6034966, 936355]","[1082185, 981760, 999999, 1127831, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[826790, 1070898, 1098248, 828867, 1125197]","[944139, 1131438, 5981072, 889509, 930917]","[999999, 903738, 5996227, 10150194, 856335]","[999999, 1082185, 981760, 1127831, 1098066]","[999999, 6534178, 6533889, 1029743, 6534166]","[839849, 12782182, 865528, 946757, 7025250]","[1082185, 981760, 999999, 1098066, 826249]"


##### 3.4. Ансамбль Popularity Recommender и Random Recommender 

In [47]:
def get_union_2():
    rec_1 = set(popularity_recommendation(data_train, n=5))
    rec_2 = set(random_recommendation(items, n=5))

    res = rec_1&rec_2
    
    while len(res) < 5:
        res.add((rec_1^rec_2).pop()) 
    
    return list(res)

In [ ]:
%%time

get_union_2()

In [ ]:
result_5k['ansamble_2'] = result_5k['user_id'].apply(lambda x:  GetUnion())    
score = result_5k.apply(lambda x: precision_at_k(x['ansamble'], x['actual'],  5), axis=1).mean()         
print(score)

##### 3.5. Ансамбль Popularity Recommendation, Random Recommendation, ItemItemRecommender и CosineRecommender 

In [ ]:
def get_union_4():
    rec_1 = set(popularity_recommendation(data_train, n=5))
    rec_2 = set(random_recommendation(items, n=5))
    rec_3 = set(itemitem_recs)
    rec_4 = set(cosine_recs)

    res = rec_1&rec_2&rec_3&rec_4
    
    while len(res) < 5:
        res.add((rec_1^rec_2^rec_3^rec_4).pop()) 
    
    return list(res)

In [ ]:
%%time

get_union_4()  

In [ ]:
result_5k['ansamble_4'] = result_5k['user_id'].apply(lambda x:  GetUnion())    
score = result_5k.apply(lambda x: precision_at_k(x['ansamble_4'], x['actual'],  5), axis=1).mean()         
print(score)

время 1:32:34